In [415]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

In [416]:
data = pd.read_csv('owid-covid-data.csv', sep=',')
data

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
0,ABW,Aruba,2020-03-13,2,2,0,0,18.733,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
1,ABW,Aruba,2020-03-20,4,2,0,0,37.465,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
2,ABW,Aruba,2020-03-24,12,8,0,0,112.395,74.930,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
3,ABW,Aruba,2020-03-25,17,5,0,0,159.227,46.831,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
4,ABW,Aruba,2020-03-26,19,2,0,0,177.959,18.733,0.0,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16796,NaN,International,2020-02-28,705,0,4,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16797,NaN,International,2020-02-29,705,0,6,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16798,NaN,International,2020-03-01,705,0,6,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16799,NaN,International,2020-03-02,705,0,6,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Подготовка данных для обучения регрессионной модели

Предсказание окончания карантина будет сложно получить предсказывая количество заражений при помощи алгоритмов временных рядов, данные алгоритмы из данных заболеваемости и увеличения числа инфецированных извлекут лишь тендецию роста, предсказание будет актуально на ближайшие дни, но абсолютно не верно в долгосрочной перспективе. Поэтому мы решаем задачу регрессии.

Окончание карантина - не является унифицированным критерием, лишь правительство стран решает когда остановить карантин, поэтому нам необходимо формализовать данный критерий. Мы не можем следовать официальной статистике даты окончания карантина, в некоторых странах в связи с какими-либо причинами он может вообще не вводиться, а датасет из стран официально вышедших из карантина будет слишком мал для обучения регрессионной модели.

Поэтому, введем формальный критерий окончания карантина - рост заболеваемости в сутки вышел на уровень не более 10 случаев заражений в сутки.

In [417]:
# Посмотрим на колиество пропусков значений в разных свойствах
data.isna().sum()

iso_code                       64
location                        0
date                            0
total_cases                     0
new_cases                       0
total_deaths                    0
new_deaths                      0
total_cases_per_million       309
new_cases_per_million         309
total_deaths_per_million      309
new_deaths_per_million        309
total_tests                 12659
new_tests                   13093
total_tests_per_thousand    12659
new_tests_per_thousand      13084
tests_units                 12659
population                    230
population_density            706
median_age                   1455
aged_65_older                1656
aged_70_older                1531
gdp_per_capita               1646
extreme_poverty              6698
cvd_death_rate               1424
diabetes_prevalence           970
female_smokers               4119
male_smokers                 4248
handwashing_facilities      10257
hospital_beds_per_100k       2517
dtype: int64

In [418]:
# Удалим свойства с большим количеством пропущенных значений
data = data.drop(['total_tests', 'new_tests', 'total_tests_per_thousand', 'new_tests_per_thousand', 'tests_units', 'handwashing_facilities', 'extreme_poverty'], 1)

In [419]:
# Удалим свойство iso_code(iso код страны), т.к. он не вносит никакой значимой информации
data = data.drop(['iso_code'], 1)

# Доля данных с пропусками не очень велика, можно выбросить оставшиеся объекты с пропусками
data = data.dropna()
data = data.reset_index(drop=True)

In [420]:
# Рассмотрим корреляцию между парами свойств
data.corr().style.background_gradient(cmap='coolwarm')

,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_100k
total_cases,1.000000,0.901613,0.988322,0.858802,0.127443,0.080851,0.148922,0.112079,0.523126,-0.020503,0.009818,0.014601,0.012915,0.001606,-0.032407,0.022576,0.004537,-0.000181,-0.008961
new_cases,0.901613,1.000000,0.862311,0.951316,0.091461,0.118466,0.093903,0.126378,0.592092,-0.022652,0.003802,0.010036,0.007813,0.002246,-0.031131,0.028764,0.004173,-0.002686,-0.013338
total_deaths,0.988322,0.862311,1.000000,0.836049,0.150241,0.078567,0.226181,0.150178,0.493754,-0.020250,0.019201,0.028038,0.027660,0.001382,-0.042910,0.005369,0.015536,-0.005504,-0.012110
new_deaths,0.858802,0.951316,0.836049,1.000000,0.114893,0.113882,0.157781,0.216307,0.550332,-0.023190,0.016110,0.027183,0.026178,0.003624,-0.046958,0.009947,0.016502,-0.010885,-0.015732
total_cases_per_million,0.127443,0.091461,0.150241,0.114893,1.000000,0.602167,0.679470,0.500301,-0.041296,0.042165,0.224862,0.205674,0.211609,0.330818,-0.231139,-0.037606,0.208686,-0.101610,0.053037
new_cases_per_million,0.080851,0.118466,0.078567,0.113882,0.602167,1.000000,0.307395,0.462411,-0.037773,0.084728,0.170619,0.122206,0.125667,0.325614,-0.183862,0.035027,0.134817,-0.081229,0.025167
total_deaths_per_million,0.148922,0.093903,0.226181,0.157781,0.679470,0.307395,1.000000,0.668506,-0.018784,-0.018726,0.208193,0.234657,0.244669,0.147624,-0.193558,-0.106649,0.217052,-0.068045,0.042692
new_deaths_per_million,0.112079,0.126378,0.150178,0.216307,0.500301,0.462411,0.668506,1.000000,-0.019281,-0.021509,0.201936,0.230631,0.238775,0.151754,-0.193648,-0.107779,0.216168,-0.077753,0.041261
population,0.523126,0.592092,0.493754,0.550332,-0.041296,-0.037773,-0.018784,-0.019281,1.000000,-0.027351,-0.056576,-0.059682,-0.066952,-0.088149,0.012275,0.048067,-0.091455,0.034685,-0.044041
population_density,-0.020503,-0.022652,-0.020250,-0.023190,0.042165,0.084728,-0.018726,-0.021509,-0.027351,1.000000,0.106828,0.007138,-0.026240,0.291900,-0.167938,0.175534,-0.075600,-0.011576,-0.034258


In [421]:
# Видно, что очень высока корреляция между свойствами aged_65_older и aged_70_older. Действительно, одно свойство практически однозначно
# выражается через другое. Одно из свойств необходимо удалить из датасета

# Высокую корреляцию имеют свойства total_cases(всего зараженных) и total_deaths(всего смертей), а также new_cases(новые заражения) и new_deaths(новые смерти)

data = data.drop(['aged_70_older', 'total_deaths', 'new_deaths'], 1)

Обработаем категориальные признаки для обучения линейной регрессии. Воспользуемся dummy(one-hot) кодированием

In [422]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

new_ohe_features = ohe.fit_transform(data['location'].values.reshape(-1, 1))
tmp = pd.DataFrame(new_ohe_features, columns=['location=' + str(i) for i in range(new_ohe_features.shape[1])])

data = pd.concat([data.drop(['location'], axis=1), tmp], axis=1)

In [423]:
# Преобразуем даты в числа

import datetime as dt
data['date'] = pd.to_datetime(data['date'])
data['date'] = data['date'].map(dt.datetime.toordinal)

Наш датасет содержит большое количество исторических данных, тоесть данные представляют собой временной ряд. Очевидно, что мы не можем использовать такие данные в регрессионной модели, но в таком случае мы будем терять огромное количество информации. Поэтому преобразуем все записи, относящиеся к одной стране в один объект, в который добавим свойства значений роста числа инфецированных в день со старта объявления карантина в течение 30 дней.

In [424]:
import re

pettern = re.compile("^location=[\d]+")
countries_list = []

for column_name in data.columns.values:
    if pattern.match(column_name):
        countries_list.append(column_name)

print(countries_list[0:10])

        
# Для обучения регрессии нам нужны примеры стран в которых был введен, а затем отменен карантин. Такими странами будем считать те, в которых
# прирост заболевающих в сутки превышал 10 хотя бы единожды, а на текущий момент прирост в предыдущий день ниже этого значения.

# Найдем список таких стран, индексы интервала данных для этой страны, а также индексы дней старта и окончания карантина
emergency_ended = {}

for country in countries_list:
    country_data = data.loc[data[country] == 1]
    new_cases_history = country_data.new_cases.values
    if len([ i for (i,j) in enumerate(new_cases_history) if j >= 10 ]) != 0 and new_cases_history[-1] < 10:
        higher_than_300 = [ i for (i,j) in enumerate(new_cases_history) if j >= 10 ]
        emergency_ended[country] = (country_data.index[0], country_data.index[-1], country_data.index[higher_than_300[0]], country_data.index[higher_than_300[-1]])
        
# Отбрасываем страны где карантин длился меньше 30 дней
filtered_emergency_ended_list = {}
for key in emergency_ended.keys():
    if (emergency_ended[key][3] - emergency_ended[key][2]) > 30:
        filtered_emergency_ended_list[key] = emergency_ended[key]
        

# Финальный список стран для обучения модели, ключ - закодированное название страны, хранимые данные - первый и последний индексы строк данных 
# относящихся к этой стране, первый и последний индексы строк периода карантина
filtered_emergency_ended_list

['location=0', 'location=1', 'location=2', 'location=3', 'location=4', 'location=5', 'location=6', 'location=7', 'location=8', 'location=9']


{'location=0': (0, 64, 4, 63),
 'location=4': (385, 518, 450, 517),
 'location=19': (971, 1031, 975, 1026),
 'location=24': (2163, 2296, 2163, 2295),
 'location=27': (2424, 2489, 2435, 2488),
 'location=28': (4727, 4857, 4800, 4856),
 'location=29': (2490, 2551, 2495, 2538),
 'location=34': (3263, 3391, 3335, 3390),
 'location=38': (3707, 3835, 3775, 3824),
 'location=44': (4348, 4478, 4437, 4472),
 'location=50': (5506, 5639, 5570, 5617),
 'location=64': (7277, 7343, 7288, 7340),
 'location=67': (7025, 7149, 7096, 7148),
 'location=68': (7150, 7276, 7217, 7273),
 'location=71': (7598, 7645, 7605, 7644),
 'location=72': (7646, 7709, 7655, 7701),
 'location=85': (8340, 8473, 8405, 8472),
 'location=99': (10079, 10144, 10084, 10141),
 'location=100': (10145, 10212, 10150, 10201),
 'location=104': (6900, 7024, 6968, 7023),
 'location=110': (10525, 10651, 10591, 10650),
 'location=112': (10465, 10524, 10489, 10523),
 'location=113': (10704, 10768, 10715, 10759),
 'location=115': (10886, 10

In [425]:
# Сформируем финальную таблицу данных для обучения модели
X = pd.DataFrame().reindex_like(data).drop(data.index).drop(['date', 'total_cases', 'new_cases', 'total_cases_per_million',
                                                              'new_cases_per_million', 'total_deaths_per_million', 'new_deaths_per_million'], 1)

# Добавляем столбцы для данных первых 30 дней после введения карантина
for x in range(30):
    X["Day={}".format(x)] = []


y = [ value[3] - value[2] for value in filtered_emergency_ended_list.values() ]

X.columns.values

array(['population', 'population_density', 'median_age', 'aged_65_older',
       'gdp_per_capita', 'cvd_death_rate', 'diabetes_prevalence',
       'female_smokers', 'male_smokers', 'hospital_beds_per_100k',
       'location=0', 'location=1', 'location=2', 'location=3',
       'location=4', 'location=5', 'location=6', 'location=7',
       'location=8', 'location=9', 'location=10', 'location=11',
       'location=12', 'location=13', 'location=14', 'location=15',
       'location=16', 'location=17', 'location=18', 'location=19',
       'location=20', 'location=21', 'location=22', 'location=23',
       'location=24', 'location=25', 'location=26', 'location=27',
       'location=28', 'location=29', 'location=30', 'location=31',
       'location=32', 'location=33', 'location=34', 'location=35',
       'location=36', 'location=37', 'location=38', 'location=39',
       'location=40', 'location=41', 'location=42', 'location=43',
       'location=44', 'location=45', 'location=46', 'location=47',

In [426]:
for index, location in enumerate(filtered_emergency_ended_list.values()):
    new_line = data.loc[location[0]][['population', 'population_density', 'median_age', 'aged_65_older', 'gdp_per_capita', 'cvd_death_rate', 'diabetes_prevalence', 
                                     'female_smokers', 'male_smokers', 'hospital_beds_per_100k', *[ "location={}".format(x) for x in range(0, 127) ]]].values
    
    for x in range(location[2], location[2] + 30, 1):
        new_line = np.append(new_line, data.loc[x]['new_cases'])
    
    X.loc[index] = new_line

Нормализация данных важна при использовании алгоритмов на основе градиентного спуска. В нашем наборе данных видно, что масштаб разных свойств сильно отличается, например, значения свойства  популяция(до 10 ** 9) и процент курящих.

In [427]:
# Нормализуем данные

X_scaled = (X - X.mean()) / (X.max() - X.min())
X_scaled = X_scaled.fillna(0)

In [428]:
from sklearn.model_selection import train_test_split

# Делим данные на обучучающую и тестирующую выборки
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, train_size=0.7, test_size=0.3, shuffle=True)

In [429]:
# Конечный вид данных для обучения модели
X_train

,population,population_density,median_age,aged_65_older,gdp_per_capita,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_100k,...,Day=20,Day=21,Day=22,Day=23,Day=24,Day=25,Day=26,Day=27,Day=28,Day=29
23,-0.015497,0.043785,-0.647160,-0.560981,-0.231606,-0.028596,-0.454085,-0.236383,-0.294912,-0.409844,...,-0.106358,-0.166864,-0.180846,-0.158835,-0.147789,-0.062455,-0.090671,-0.090404,-0.061524,-0.064448
12,-0.045392,-0.072440,0.317253,0.396258,0.068094,0.305201,-0.312438,0.290087,0.103219,0.528236,...,-0.053025,-0.052502,-0.060955,-0.042095,-0.134196,-0.060175,-0.110220,-0.064991,-0.028438,-0.005785
6,-0.046676,-0.015561,0.096613,0.078619,0.099229,-0.214376,0.361896,0.240087,0.377985,0.039072,...,-0.073025,-0.116332,-0.101827,-0.090553,-0.077886,-0.049687,-0.078641,-0.070074,-0.044981,-0.044667
26,-0.034509,-0.087701,-0.600896,-0.543239,-0.210156,0.025895,-0.279751,-0.245207,-0.145379,-0.177646,...,-0.099691,-0.153566,-0.169947,-0.152227,-0.151672,-0.059719,-0.107212,-0.081510,-0.061524,-0.064448
0,-0.045284,-0.031267,0.121524,0.065654,-0.122764,0.205327,0.463591,-0.127560,0.349948,-0.039875,...,-0.079691,-0.118991,-0.107276,-0.097161,-0.097303,-0.056527,-0.101197,-0.070074,-0.056390,-0.062402
10,-0.047047,-0.101204,0.096613,0.136335,0.250743,-0.274050,-0.113891,0.084205,-0.322949,-0.036779,...,-0.073025,-0.007289,0.061661,-0.017866,0.019202,-0.030535,-0.024506,-0.007812,-0.033572,-0.009196
3,0.952953,-0.001764,0.146435,-0.079177,-0.085012,0.096436,0.422428,-0.280501,0.297612,0.184583,...,-0.074691,0.234732,0.200626,0.052619,0.351241,0.137272,0.889780,0.908325,0.938476,0.932823
11,-0.045974,-0.082037,0.331488,0.439019,0.020054,0.323405,-0.162317,0.416558,0.346210,0.374986,...,-0.091358,-0.150906,-0.101827,-0.134606,-0.107012,-0.055615,-0.078641,-0.086592,-0.060383,-0.061038
16,-0.043517,-0.093582,0.182022,0.272239,0.448024,-0.283514,-0.113891,0.240087,-0.220145,0.070032,...,0.218642,0.763987,0.473105,0.775086,0.360949,0.053368,0.076246,0.189138,0.062834,0.116998
4,-0.043744,-0.037327,-0.035060,-0.145878,-0.082682,-0.222610,0.306206,-0.148148,-0.281828,-0.312321,...,-0.079691,-0.116332,-0.080028,-0.119187,-0.114779,-0.055615,-0.084656,-0.084051,-0.056961,-0.056263


# Обучение модели

In [430]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Обучаем линейную регрессию
reg = LinearRegression().fit(X_train, y_train)

In [431]:
reg.coef_

array([ 1.12551434e+01, -1.67419676e-01,  3.37377320e+00, -1.20689358e+00,
       -1.39469458e+00, -3.49321734e+00,  7.00462209e+00, -4.23518817e+00,
       -2.51738327e-01,  5.36096620e+00,  8.59503260e+00,  1.77635684e-15,
        4.44089210e-16, -8.88178420e-16,  8.34634165e-16,  1.77635684e-15,
        4.44089210e-16,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  3.40816326e-21,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  1.10805019e+01,  0.00000000e+00,
        0.00000000e+00,  1.15927018e-01,  3.40816326e-21, -5.55229595e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        3.40816326e-21,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        3.40816326e-21,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [432]:
# Ни один из коэффициентов не имеет больших значений, поэтому не очевидно, переобучилась ли модель
# Проверим ошибку на обучающей и тестовой выборках

print('linear regression without regularization')
print('X_train MSE:', mean_squared_error(y_train, reg.predict(X_train)))
print('X_train MSE:', mean_squared_error(y_test, reg.predict(X_test)))

linear regression without regularization
X_train MSE: 5.441387221791157e-28
X_train MSE: 39.02093326934964


In [433]:
# Модель имеет огромную разницу ошибки на обучающей и тестовой выборках. Можно предположить, что она переобучилась
# Попробуем обучить линейную регрессию с L2 регуляризацией:

In [434]:
from sklearn.linear_model import Ridge

reg = Ridge().fit(X_train, y_train)
reg.coef_

array([ 9.64632743e+00, -1.51057237e-01,  2.95084493e+00, -5.36821507e-01,
       -6.91742459e-01, -2.37867744e+00,  5.75373665e+00, -3.09824590e+00,
        3.90796081e-01,  3.73958044e+00,  4.09442116e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  8.01186857e-32,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  8.01186857e-32,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  9.58583907e+00,  0.00000000e+00,
        0.00000000e+00,  3.21334490e-01,  8.01186857e-32, -3.22513585e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        8.01186857e-32,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        8.01186857e-32,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [435]:
print('Linear regression with L2 regularization')
print('X_train MSE:', mean_squared_error(y_train, reg.predict(X_train)))
print('X_train MSE:', mean_squared_error(y_test, reg.predict(X_test)))

Linear regression with L2 regularization
X_train MSE: 10.30048650128235
X_train MSE: 35.076172536499904


In [436]:
# Видно, что разрыв между качеством модели на обучающей и тестовой выборках теперь не такой значительный